# /query and /predicate functionality for CHP

Queries for our system are in the probabilistic form of:<br>
$P(survival\_time > X | Mut\_g_1 = True, Mut\_g_2 = True, ..., Mut\_g_n = True, Drug = d_1, Drug = d_2, ..., Drug = d_n)$<br>
Returned is a knowledge graph containing probability of survival time and contribution analysis (if specified)

In [39]:
import requests
import json
import csv

In [52]:
r = requests.get('http://chp.thayer.dartmouth.edu/predicates/')
print(r.content)

b'{"gene": {"disease": ["gene_to_disease_association"]}, "drug": {"disease": ["chemical_to_disease_or_phenotypic_feature_association"]}, "disease": {"phenotypicfeature": ["disease_to_phenotypic_association"]}}'


In [41]:
# Function: buildQuery
#
# Input:
# -----------
# list of gene and drug tuples
#
# Output:
# -----------
# A query graph that answers this probablistic question: 
# P(survival_time > X | Mut_g1 = True, Mut_g2 = True, ..., Mut_gn = True, Drug = d1, Drug = d2, ..., Drug = dn)

def buildQuery(st, disease, genes=[], drugs=[]):
    
    # empty response
    reasoner_std = { "query_graph": dict()}
    
    # empty query graph
    reasoner_std["query_graph"] = { "edges": dict(),
                                    "nodes": dict()
                                  }
    
    node_count = 0
    edge_count = 0
    
    # add genes
    for gene in genes:
        reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category':'gene',
                                                                           'curie':'{}'.format(gene[1])
                                                                         }
        node_count += 1
    
    # add drugs
    for drug in drugs:
        reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category':'drug',
                                                                           'curie':'{}'.format(drug[1])
                                                                         }
        node_count += 1
    
    # add in disease node
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category':'disease',
                                                                       'curie':'{}'.format(disease[1])
                                                                     }
    node_count += 1
    
    # link all evidence to disease
    for node_id in reasoner_std['query_graph']['nodes'].keys():
        node = reasoner_std['query_graph']['nodes'][node_id]
        if node['category'] == 'gene':
            reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'predicate':'gene_to_disease_association',
                                                                               'subject': node_id,
                                                                               'object': 'n{}'.format(node_count -1)   # should be disease node
                                                                             }
            edge_count += 1
        elif node['category'] == 'drug':
            reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'predicate':'chemical_to_disease_or_phenotypic_feature_association',
                                                                               'subject': node_id,
                                                                               'object': 'n{}'.format(node_count -1)  # should be disease node
                                                                             }
            edge_count += 1
            
    # add target survival node
    phenotype = ('Survival_Time', 'EFO:0000714')
    reasoner_std['query_graph']['nodes']['n{}'.format(node_count)] = { 'category': 'phenotypicfeature',
                                                                       'curie': '{}'.format(phenotype[1]),
                                                                     }
    node_count += 1
    
    # link disease to target
    reasoner_std['query_graph']['edges']['e{}'.format(edge_count)] = { 'predicate':'disease_to_phenotype_association',
                                                                       'subject': 'n{}'.format(node_count-2),
                                                                       'object': 'n{}'.format(node_count-1),
                                                                       'properties': { 'qualifier':'>=',
                                                                                       'days': st
                                                                                     }
                                                                     }
    return reasoner_std

In [42]:
def readGenes():
    with open('gene_curie_map.csv', 'r') as gene_file:
        reader = csv.reader(gene_file)
        next(reader)
        rows = [(row[0],row[1]) for row in reader]
    return rows

In [43]:
def readDrugs():
    with open('drug_curie_map.csv', 'r') as drug_file:
        reader = csv.reader(drug_file)
        next(reader)
        rows = [(row[0],row[1]) for row in reader]
    return rows

In [54]:
# list of genes (and curies) we can query over
#gene_list = readGenes()

# list of drugs (and curies) we can query over
#drug_list = readDrugs()

# or pull from above lists
genes = [('RAF1', 'ENSEMBL:ENSG00000132155'),
         ('MAP3K13','ENSEMBL:ENSG00000073803')]
drugs = [('CYCLOPHOSPHAMIDE', 'CHEMBL:CHEMBL88')]
disease = ('Breast_Cancer', 'MONDO:0007254')
survival_time = 1000
# if genes or drugs is an empty list you can omit them
query = buildQuery(survival_time, disease, genes=genes, drugs=drugs)
payload = {'message': query}
r = requests.post('http://chp.thayer.dartmouth.edu/query/', json=payload)
chp_res = json.loads(r.content)
KG = reasoner_std_final['message']['knowledge_graph']
for edge_key in KG['edges'].keys():
    edge = KG['edges'][edge_key]
    if edge['predicate'] == 'disease_to_phenotype_association':
        p_survival = edge['has_confidence_level']

# probability of surival given QG specification
print("Probability of survival > {} days is:".format(survival_time), p_survival)

b'\n<!doctype html>\n<html lang="en">\n<head>\n  <title>Server Error (500)</title>\n</head>\n<body>\n  <h1>Server Error (500)</h1><p></p>\n</body>\n</html>\n'


JSONDecodeError: Expecting value: line 2 column 1 (char 1)